In [345]:
"""
Minimal test for solve functions (OPF, PF preprocessing, PF post-processing)
Tests the complete workflow on several IEEE cases
"""

import numpy as np
from gridfm_datakit.network import load_net_from_pglib
from gridfm_datakit.process.solvers import run_opf, run_dcpf
from gridfm_datakit.process.process_network import (
    init_julia,
    pf_preprocessing,
)
from gridfm_datakit.utils.idx_gen import GEN_BUS

In [346]:
case_name = "case300_ieee"

In [347]:
jl = init_julia(150)

In [348]:
net = load_net_from_pglib(case_name)
n_buses = net.buses.shape[0]
print("\n  ====== PART 2: Testing AC OPF + DC PF ======")

# Run AC OPF
print("  Running AC OPF...")
opf_result = run_opf(net, jl)
assert str(opf_result["termination_status"]) == "LOCALLY_SOLVED", (
    "AC OPF should converge"
)
print("AC OPF converged successfully")

# Get generator Pg from AC OPF (before PF)
pg_before_pf = {}
for i in net.idx_gens_in_service:
    pg_before_pf[i] = opf_result["solution"]["gen"][str(i + 1)]["pg"] * net.baseMVA

# PF preprocessing
print("  Running PF preprocessing...")
net = pf_preprocessing(net, opf_result)
print("PF preprocessing completed")

# Run DC PF
print("  Running DC PF...")
dcpf_result = run_dcpf(net, jl, fast=True)


  ====== PART 2: Testing AC OPF + DC PF ======
  Running AC OPF...
AC OPF converged successfully
  Running PF preprocessing...
PF preprocessing completed
  Running DC PF...


In [349]:
dcpf_result["solution"]

Dict{String, Any} with 4 entries:
  "branch"   => Dict{String, Any}("306"=>Dict("qf"=>NaN, "qt"=>NaN, "pf"=>-3.28…
  "bus"      => Dict{String, Any}("1"=>Dict("va"=>0.343084), "54"=>Dict("va"=>0…
  "per_unit" => true
  "pf"       => true

In [351]:
pg_gen = np.array(
    [
        opf_result["solution"]["gen"][str(i + 1)]["pg"] * net.baseMVA
        for i in net.idx_gens_in_service
    ],
)
qg_gen = np.array(
    [
        opf_result["solution"]["gen"][str(i + 1)]["qg"] * net.baseMVA
        for i in net.idx_gens_in_service
    ],
)
gen_bus = net.gens[net.idx_gens_in_service, GEN_BUS].astype(int)
Pg_bus = np.bincount(gen_bus, weights=pg_gen, minlength=n_buses)
Qg_bus = np.bincount(gen_bus, weights=qg_gen, minlength=n_buses)

In [352]:
from gridfm_datakit.utils.idx_brch import F_BUS, T_BUS

In [353]:
pf_dcpf = np.array(
    [
        dcpf_result["solution"]["branch"][str(i + 1)]["pf"] * net.baseMVA
        for i in net.idx_branches_in_service
    ],
)

pt_dcpf = np.array(
    [
        dcpf_result["solution"]["branch"][str(i + 1)]["pt"] * net.baseMVA
        for i in net.idx_branches_in_service
    ],
)

In [354]:
pd_slack = net.Pd[net.ref_bus_idx]
print("pd_slack", pd_slack)
pg_slack = Pg_bus[net.ref_bus_idx]
print("pg_slack", pg_slack)

# branches with slack as from/to bus
branches_from = net.branches[net.idx_branches_in_service, F_BUS] == net.ref_bus_idx
print("branches_from", np.array(branches_from))
branches_to = net.branches[net.idx_branches_in_service, T_BUS] == net.ref_bus_idx
print("branches_to", branches_to)

sum_flows_from = pf_dcpf[branches_from].sum()
print("sum_flows_from", sum_flows_from)
sum_flows_to = pt_dcpf[branches_to].sum()
print("sum_flows_to", sum_flows_to)

# power balance at slack
balance = pg_slack - pd_slack - (sum_flows_from + sum_flows_to)
print("balance", balance)

# find generators at slack bus
slack_gen = np.where(net.gens[net.idx_gens_in_service, GEN_BUS] == net.ref_bus_idx)[0]
print("slack_gen", slack_gen)

# copy current setpoints
pg_gen_dc = pg_gen.copy()

# assign entire balance to first generator at slack
first_slack_gen = slack_gen[0]
pg_gen_dc[first_slack_gen] -= balance

pd_slack 0.0
pg_slack 496.3416145926895
branches_from [False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False Fal

In [355]:
assert np.allclose(
    pg_gen_dc[[i for i in net.idx_gens_in_service if not np.isin(i, slack_gen)]],
    pg_gen[[i for i in net.idx_gens_in_service if not np.isin(i, slack_gen)]],
)

In [356]:
print(pg_gen_dc)
slack_gen = np.where(net.gens[net.idx_gens_in_service, GEN_BUS] == net.ref_bus_idx)[0]
print(pg_gen_dc[slack_gen])

[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  8.20407821e+02  7.04134395e-06  6.10487031e+01
  9.30000005e+01  1.97000001e+02  1.66584000e+03  3.21000002e+02
  0.00000000e+00  0.00000000e+00  2.97000001e+02  8.00000005e+02
  9.10000004e+01 -3.93777779e-07  3.92837750e-06  2.42981786e+02
  2.42000002e+02  0.00000000e+00  2.09150390e+02  0.00000000e+00
  1.02748558e+02  1.25000000e+02  2.50000002e+02  2.46500002e+03
  1.62400001e+03  2.17720949e-07  1.49130790e+03  4.30000003e+02
  1.72387265e-07  4.60000004e+02  5.54000005e+02  3.15671887e-08
  4.36000004e+02  4.57999997e+02  3.41000002e+02  8.81000008e+02
  2.66999997e+02  4.25029815e+02  1.12700001e+03 -8.78857118e-07
  3.41999998e+02  2.79867434e-07  6.70000003e+02  1.40100001e+03
  6.37165413e+02  5.54014321e+02  4.90000003e+02 -5.58517579e-07
  4.96112850e+02 -8.50739935e-07  8.25574959e+01  6.99243211e+01
  8.78210781e+01  1.53332627e-06  5.31979164e+02  3.57114370e+02
  2.03000001e+02  8.00533

In [357]:
# solve
dcpf_result2 = run_dcpf(net, jl, fast=False)
pg_gen_dc2 = np.array(
    [
        dcpf_result2["solution"]["gen"][str(i + 1)]["pg"] * net.baseMVA
        for i in net.idx_gens_in_service
    ],
)
print(pg_gen_dc2)
print(pg_gen_dc2[slack_gen])

[ 0.00000000e+00 -1.52723471e-25  0.00000000e+00  0.00000000e+00
  0.00000000e+00  8.20407821e+02  7.04134395e-06  6.10487031e+01
  9.30000005e+01  1.97000001e+02  1.66584000e+03  3.21000002e+02
 -4.03896783e-26  0.00000000e+00  2.97000001e+02  8.00000005e+02
  9.10000004e+01 -3.93777779e-07  3.92837750e-06  2.42981786e+02
  2.42000002e+02  0.00000000e+00  2.09150390e+02  0.00000000e+00
  1.02748558e+02  1.25000000e+02  2.50000002e+02  2.46500002e+03
  1.62400001e+03  2.17720949e-07  1.49130790e+03  4.30000003e+02
  1.72387265e-07  4.60000004e+02  5.54000005e+02  3.15671887e-08
  4.36000004e+02  4.57999997e+02  3.41000002e+02  8.81000008e+02
  2.66999997e+02  4.25029815e+02  1.12700001e+03 -8.78857118e-07
  3.41999998e+02  2.79867434e-07  6.70000003e+02  1.40100001e+03
  6.37165413e+02  5.54014321e+02  4.90000003e+02 -5.58517579e-07
  4.96112850e+02 -8.50739935e-07  8.25574959e+01  7.25243211e+01
  8.78210781e+01  1.53332627e-06  5.31979164e+02  3.57114370e+02
  2.03000001e+02  8.00533

In [362]:
pf_dcpf2 = np.array(
    [
        dcpf_result2["solution"]["branch"][str(i + 1)]["pf"] * net.baseMVA
        for i in net.idx_branches_in_service
    ],
)

pt_dcpf2 = np.array(
    [
        dcpf_result2["solution"]["branch"][str(i + 1)]["pt"] * net.baseMVA
        for i in net.idx_branches_in_service
    ],
)

In [358]:
assert np.allclose(
    pg_gen_dc[[i for i in net.idx_gens_in_service if not np.isin(i, slack_gen)]],
    pg_gen_dc2[[i for i in net.idx_gens_in_service if not np.isin(i, slack_gen)]],
)

In [361]:
pg_gen_dc - pg_gen_dc2

array([ 0.00000000e+00,  1.52723471e-25,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        4.03896783e-26,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  

In [364]:
pd_slack = net.Pd[net.ref_bus_idx]
print("pd_slack", pd_slack)
pg_slack = Pg_bus[net.ref_bus_idx]
print("pg_slack", pg_slack)

# branches with slack as from/to bus
branches_from = net.branches[net.idx_branches_in_service, F_BUS] == net.ref_bus_idx
branches_to = net.branches[net.idx_branches_in_service, T_BUS] == net.ref_bus_idx


sum_flows_from = pf_dcpf[branches_from].sum()
print("sum_flows_from", sum_flows_from)
sum_flows_from2 = pf_dcpf2[branches_from].sum()
print("sum_flows_from2", sum_flows_from2)


sum_flows_to = pt_dcpf[branches_to].sum()
print("sum_flows_to", sum_flows_to)
sum_flows_to2 = pt_dcpf2[branches_to].sum()
print("sum_flows_to2", sum_flows_to2)

# power balance at slack
balance = pg_slack - pd_slack - (sum_flows_from + sum_flows_to)
print("balance", balance)

# find generators at slack bus
slack_gen = np.where(net.gens[net.idx_gens_in_service, GEN_BUS] == net.ref_bus_idx)[0]
print("slack_gen", slack_gen)

pd_slack 0.0
pg_slack 496.3416145926895
sum_flows_from 69.92432113495354
sum_flows_from2 72.52432113494044
sum_flows_to 0.0
sum_flows_to2 0.0
balance 426.4172934577359
slack_gen [55]


In [365]:
dcpf_result2["solution"]

Dict{String, Any} with 8 entries:
  "baseMVA"             => 100
  "gen"                 => Dict{String, Any}("32"=>Dict{String, Any}("qg"=>NaN,…
  "branch"              => Dict{String, Any}("306"=>Dict("qf"=>NaN, "qt"=>NaN, …
  "multiinfrastructure" => false
  "multinetwork"        => false
  "bus"                 => Dict{String, Any}("1"=>Dict{String, Any}("va"=>0.341…
  "per_unit"            => true
  "pf"                  => true

In [375]:
dcpf_result["solution"]["bus"]

Dict{String, Any} with 300 entries:
  "1"    => Dict("va"=>0.343084)
  "54"   => Dict("va"=>0.0124349)
  "41"   => Dict("va"=>0.0221465)
  "9033" => Dict("va"=>-0.245663)
  "168"  => Dict("va"=>0.266127)
  "159"  => Dict("va"=>0.0682826)
  "228"  => Dict("va"=>0.221739)
  "190"  => Dict("va"=>0.173177)
  "227"  => Dict("va"=>0.121679)
  "223"  => Dict("va"=>0.133658)
  "531"  => Dict("va"=>0.104731)
  "7012" => Dict("va"=>0.587521)
  "88"   => Dict("va"=>0.124305)
  "26"   => Dict("va"=>0.213558)
  "250"  => Dict("va"=>0.119649)
  "77"   => Dict("va"=>0.196457)
  "230"  => Dict("va"=>0.396563)
  "24"   => Dict("va"=>0.416734)
  "7003" => Dict("va"=>0.570776)
  ⋮      => ⋮

In [380]:
va = np.array(
    [
        dcpf_result["solution"]["bus"][str(net.reverse_bus_index_mapping[i])]["va"]
        for i in range(net.buses.shape[0])
    ]
)
va2 = np.array(
    [
        dcpf_result2["solution"]["bus"][str(net.reverse_bus_index_mapping[i])]["va"]
        for i in range(net.buses.shape[0])
    ]
)


# compare va and va2
va - va2

array([ 0.00143857,  0.00144083,  0.00143761,  0.00143697,  0.00143868,
        0.00144027,  0.00143789,  0.00144432,  0.00143985,  0.00143976,
        0.00144095,  0.00143905,  0.00144088,  0.00145247,  0.00146074,
        0.00143453,  0.00146074,  0.00144281,  0.00144081,  0.00144078,
        0.0014408 ,  0.00144079,  0.00144078,  0.00144079,  0.0014408 ,
        0.0014408 ,  0.00145181,  0.00142047,  0.00140882,  0.00140885,
        0.00159716,  0.00145911,  0.00141643,  0.00149204,  0.0013893 ,
        0.00141643,  0.00141281,  0.00139886,  0.00140368,  0.00141103,
        0.00140759,  0.00147012,  0.0003224 ,  0.00093941,  0.00111107,
        0.00139155,  0.00135153,  0.00123691,  0.00131731,  0.00135164,
        0.00136103,  0.00139616,  0.00136532,  0.00138435,  0.00137099,
        0.00137448,  0.00141074,  0.00140842,  0.00140842,  0.00140878,
        0.00140813,  0.00140757,  0.00140874,  0.00140897,  0.00140897,
        0.00140898,  0.00140975,  0.00140981,  0.00140897,  0.00